# KampInceptionResNet

## Import Libarary

In [1]:
import torch
from torch.utils.data import DataLoader

from sklearn.metrics import f1_score, confusion_matrix, classification_report

from kamp.preprocess import KampDataLoader
from kamp.utils import KampDataset, evaluate_model
from kamp.model import KampInceptionResNet

In [ ]:
help(KampDataLoader)

## Load Data

In [ ]:
DATA_PATH = './data/경진대회용 주조 공정최적화 데이터셋.csv'

data_loader = KampDataLoader(
    path = DATA_PATH,

    # 처리 안한게 더 좋았음
    # 처리 안한 것 : 0.944
    # 처리 한 것 : 최대 0.922
    do_count_trend=False,
    drop_count=False,

    get_useful_p_data=True,
    p_threshold=0.05,

    outlier_method='iso',
    iso_outlier_rate=0.0075,

    do_resample=True,
    downsampled_pass_rate=1.0,
    upsampled_fail_rate_about_pass=0.25,
    upsample_method='adasyn',
)

data_loader.process()

data = data_loader.load()

x_train = data['train_data']
y_train = data['train_label']
x_test = data['test_data']
y_test=  data['test_label']

## Modeling

In [4]:
RANDOM_SEED = 42
BATCH_SIZE = 32
EPOCHS = 1

torch.manual_seed(RANDOM_SEED)
torch.set_num_threads(8)

In [5]:
train_dataset = KampDataset(inputs=x_train, labels=y_train)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

In [ ]:
model, history = KampInceptionResNet().fit(train_dataloader)

## Evaluation

In [355]:
test_dataset = KampDataset(inputs=x_test, labels=y_test)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [ ]:
true_labels, predicted_labels = evaluate_model(model, test_dataloader)

f1 = f1_score(true_labels, predicted_labels, average='weighted', zero_division=1)
print(f"F1 Socre : {f1:.4f}\n")

conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix : ")
print(conf_matrix, '\n')

print("Classification Report : ")
print(classification_report(true_labels, predicted_labels), '\n')